# Estimating TRFs for decoder model ot reconstruct envelope from EEG

In [1]:
import sys
# Add parent directory to path to import file_paths module
sys.path.insert(0, '..')
import file_paths
import helper_functions

import eelbrain

In [2]:
# Loop through subjects to estimate TRFs
SUBJECTS = helper_functions.get_subjects()

for SUBJECT in SUBJECTS:
    print("-" * 50)
    print(f"Processing subject {SUBJECT}...")

    # Check if all TRF files already exist
    trf_dir = file_paths.TRF_DIR / SUBJECT
    trf_attended_path = trf_dir / f"{SUBJECT}_attended_decoder_trf.pickle"
    trf_unattended_path = trf_dir / f"{SUBJECT}_unattended_decoder_trf.pickle"
    trf_padded_attended_path = trf_dir / f"{SUBJECT}_padded_attended_decoder_trf.pickle"
    trf_padded_unattended_path = trf_dir / f"{SUBJECT}_padded_unattended_decoder_trf.pickle"
    
    if all([trf_attended_path.exists(), trf_unattended_path.exists(), 
            trf_padded_attended_path.exists(), trf_padded_unattended_path.exists()]):
        print(f"All TRF files for {SUBJECT} already exist, skipping.")
        continue

    # Load EEG data from pickle files
    eeg_path = file_paths.EEG_DIR / SUBJECT / f"{SUBJECT}_eeg.pickle"
    padded_eeg_path = file_paths.EEG_DIR / SUBJECT / f"{SUBJECT}_padded_eeg.pickle"
    eeg = eelbrain.load.unpickle(eeg_path)
    padded_eeg = eelbrain.load.unpickle(padded_eeg_path)

    # Load envelope data (all 4 versions)
    attended_envelope_path = file_paths.ENVELOPES_DIR / SUBJECT / f"{SUBJECT}_attended_envelope.pickle"
    unattended_envelope_path = file_paths.ENVELOPES_DIR / SUBJECT / f"{SUBJECT}_unattended_envelope.pickle"
    padded_attended_envelope_path = file_paths.ENVELOPES_DIR / SUBJECT / f"{SUBJECT}_padded_attended_envelope.pickle"
    padded_unattended_envelope_path = file_paths.ENVELOPES_DIR / SUBJECT / f"{SUBJECT}_padded_unattended_envelope.pickle"
    
    attended_envelope = eelbrain.load.unpickle(attended_envelope_path)
    unattended_envelope = eelbrain.load.unpickle(unattended_envelope_path)
    padded_attended_envelope = eelbrain.load.unpickle(padded_attended_envelope_path)
    padded_unattended_envelope = eelbrain.load.unpickle(padded_unattended_envelope_path)

    print(f"EEG: {eeg}")
    print(f"Padded EEG: {padded_eeg}")
    print(f"Attended envelope: {attended_envelope}")
    print(f"Unattended envelope: {unattended_envelope}")
    print(f"Padded attended envelope: {padded_attended_envelope}")
    print(f"Padded unattended envelope: {padded_unattended_envelope}")

    # Estimate TRFs for attended and unattended conditions using boosting
    print("Estimating attended TRF...")
    trf_attended = eelbrain.boosting(
            attended_envelope,
            eeg,
            -1.000,
            0.100, 
        )
    
    print("Estimating unattended TRF...")
    trf_unattended = eelbrain.boosting(
            eeg,
            unattended_envelope,
            -1.000,
            0.100, 
        )
    
    print("Estimating padded attended TRF...")
    trf_padded_attended = eelbrain.boosting(
            padded_eeg,
            padded_attended_envelope,
            -1.000,
            0.100, 
        )
    
    print("Estimating padded unattended TRF...")
    trf_padded_unattended = eelbrain.boosting(
            padded_eeg,
            padded_unattended_envelope,
            -1.000,
            0.100, 
        )
    
    # Save all 4 TRFs
    trf_dir.mkdir(exist_ok=True, parents=True)
    
    eelbrain.save.pickle(trf_attended, trf_attended_path)
    eelbrain.save.pickle(trf_unattended, trf_unattended_path)
    eelbrain.save.pickle(trf_padded_attended, trf_padded_attended_path)
    eelbrain.save.pickle(trf_padded_unattended, trf_padded_unattended_path)

    print(f"Saved attended decoder TRF to {trf_attended_path}")
    print(f"Saved unattended decoder TRF to {trf_unattended_path}")
    print(f"Saved padded attended decoder TRF to {trf_padded_attended_path}")
    print(f"Saved padded unattended decoder TRF to {trf_padded_unattended_path}")

--------------------------------------------------
Processing subject S1...
EEG: <NDVar 'eeg': 192000 time, 64 sensor>
Padded EEG: <NDVar 'padded_eeg': 196200 time, 64 sensor>
Attended envelope: <NDVar 'attended': 192000 time>
Unattended envelope: <NDVar 'unattended': 192000 time>
Padded attended envelope: <NDVar 'padded_attended': 196200 time>
Padded unattended envelope: <NDVar 'padded_unattended': 196200 time>
Estimating attended TRF...
Estimating unattended TRF...
Estimating padded attended TRF...
Estimating padded unattended TRF...
Saved attended decoder TRF to /Users/sylvestereley/Data/cocoha3/TRFs/S1/S1_attended_decoder_trf.pickle
Saved unattended decoder TRF to /Users/sylvestereley/Data/cocoha3/TRFs/S1/S1_unattended_decoder_trf.pickle
Saved padded attended decoder TRF to /Users/sylvestereley/Data/cocoha3/TRFs/S1/S1_padded_attended_decoder_trf.pickle
Saved padded unattended decoder TRF to /Users/sylvestereley/Data/cocoha3/TRFs/S1/S1_padded_unattended_decoder_trf.pickle
----------